# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [3]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-0e3df91132-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Players explorative search") 

Consider the following exploratory information need:

> investigate the association football players and find the main BGPs related to this sport. Compare players per countries, awards of the players and participation in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football player

2. Identify the BGP for association football club

3. Identify the BGP for association football league

4. Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

5. Identify the BGP for the ballon d'Or award

6. Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

7. Identify the BGP of the FIFA World Cup

8. Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

9. Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).


## Task 1
Identify the BGP for association football player

In [201]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [16]:
#Using ID for Occupation to search
#[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q937857'), ('name', 'association football player')]
1


Final query for this task

In [19]:
# write your final query
# Using results from last query for BGP of Association Football Player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('name', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct

## Task 2
Identify the BGP for association football club

In [20]:
# Using ID for Association football Player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?obj ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('p', 'http://www.wi

Final query for this task

In [29]:
# write your final query
#Using Association Football 
#[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q476028 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Q

## Task 3
Identify the BGP for association football league

In [34]:
# Using Association Football ID from Task 2
#[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p ?obj wd:Q2736.
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"league"^^xsd:string))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q953425'), ('name', 'Italian association football league system')]
[('p', 'http://www.wikidata.org/entity/Q1324755'), ('name', 'Argentine football league system')]
[('p', 'http://www.wikidata.org/entity/Q20949773'), ('name', 'Zamalek SC league record by opponent')]
[('p', 'http://www.wikidata.org/entity/Q20984260'), ('name', 'Vietnamese football league system')]
[('p', 'http://www.wikidata.org/entity/Q24238546'), ('name', '2016–17 Football Superleague of Kosovo')]
[('p', 'http://www.wikidata.org/entity/Q1824674'), ('name', 'league cup')]
[('p', 'http://www.wikidata.org/entity/Q178544'), ('name', 'Swedish football league system')]
[('p', 'http://www.wikidata.org/entity/Q835610'), ('name', 'German football league system')]
[('p', 'http://www.wikidata.org/entity/Q3500681'), ('name', 'Norwegian football league system')]
[('p', 'http://www.wikidata.org/entity/Q2186316'), ('name', 'Belgian Provincial leagues')]
[('p', 'http://www.wikidata.org/en

Final query for this task

In [88]:
# write your final query
#[('p', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
     wd:Q15991303 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
9


## Task 4
Return for each european country the number of association football leagues (the result set must be a list of triples country IRI, label and #leagues).

In [91]:
# write your queries
#Using Association Foorball Leagues for searching
queryString = """
SELECT DISTINCT ?p ?name ?oname ?obj
WHERE {
   # bind something
   wd:Q15991303 ?obj ?p.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8085406'), ('name', 'Template:Infobox football league'), ('oname', "topic's main template"), ('obj', 'http://www.wikidata.org/prop/direct/P1424')]
[('p', 'http://www.wikidata.org/entity/Q26887310'), ('name', 'association football team season'), ('oname', 'different from'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('p', 'http://www.wikidata.org/entity/P1132'), ('name', 'number of participants'), ('oname', 'properties for this type'), ('obj', 'http://www.wikidata.org/prop/direct/P1963')]
[('p', 'http://www.wikidata.org/entity/P2500'), ('name', 'league level below'), ('oname', 'properties for this type'), ('obj', 'http://www.wikidata.org/prop/direct/P1963')]
[('p', 'http://www.wikidata.org/entity/P6118'), ('name', 'season ends'), ('oname', 'properties for this type'), ('obj', 'http://www.wikidata.org/prop/direct/P1963')]
[('p', 'http://www.wikidata.org/entity/P3195'), ('name', 'league points system'), ('oname', 'properties for 

In [293]:
# write your queries
#Using Association Foorball Leagues for searching
queryString = """
SELECT DISTINCT ?p ?name ?oname ?obj
WHERE {
   # bind something
    ?obj ?p wd:Q15991303.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics'), ('oname', 'Category:1994 in Swedish association football leagues'), ('obj', 'http://www.wikidata.org/entity/Q8175816')]
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('name', 'category combines topics'), ('oname', 'Category:2000 in Swedish association football leagues'), ('obj', 'http://www.wikidata.org/entity/Q8182829')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('oname', 'Uzbekistan Pro League'), ('obj', 'http://www.wikidata.org/entity/Q304228')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('oname', 'Ligue 2'), ('obj', 'http://www.wikidata.org/entity/Q2950655')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('oname', "Moldovan women's football championship"), ('obj', 'http://www.wikidata.org/entity/Q2951924')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('

In [296]:
#Usign single FootBall League for BGP and checking the country connection
queryString = """
SELECT DISTINCT ?p ?name ?oname ?obj
WHERE {
   # bind something
    wd:Q3240443 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('oname', 'country'), ('obj', 'http://www.wikidata.org/prop/direct/P17')]
[('p', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league'), ('oname', 'instance of'), ('obj', 'http://www.wikidata.org/prop/direct/P31')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football'), ('oname', 'sport'), ('obj', 'http://www.wikidata.org/prop/direct/P641')]
3


In [44]:
# Using Country ID for exploring
#[('p', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p wdt:P17 ?p.
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"Italy"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q223936'), ('name', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy')]
[('p', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/entity/Q5343710'), ('name', 'Kingdom of Italy')]
4


In [266]:
# Using specific country for exploring
#[('p', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q38 ?p ?obj.
   # get the label
   ?p sc:name ?name.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('name', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('name', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('name', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalan

In [52]:
# Using continent for exploring
#[('p', 'http://www.wikidata.org/entity/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?obj wdt:P30 ?p.
   # get the label
   ?p sc:name ?name.
   #FILTER(REGEX(?name,"Italy"^^xsd:string))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q60140'), ('name', 'Indian subcontinent')]
[('p', 'http://www.wikidata.org/entity/Q55643'), ('name', 'Oceania')]
[('p', 'http://www.wikidata.org/entity/Q19464773'), ('name', 'Northern America and Mexico')]
[('p', 'http://www.wikidata.org/entity/Q1239'), ('name', 'Indian Ocean')]
[('p', 'http://www.wikidata.org/entity/Q15'), ('name', 'Africa')]
[('p', 'http://www.wikidata.org/entity/Q18'), ('name', 'South America')]
[('p', 'http://www.wikidata.org/entity/Q4412'), ('name', 'West Africa')]
[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
[('p', 'http://www.wikidata.org/entity/Q48'), ('name', 'Asia')]
[('p', 'http://www.wikidata.org/entity/Q49'), ('name', 'North America')]
[('p', 'http://www.wikidata.org/entity/Q51'), ('name', 'Antarctica')]
[('p', 'http://www.wikidata.org/entity/Q538'), ('name', 'Insular Oceania')]
[('p', 'http://www.wikidata.org/entity/Q5401'), ('name', 'Eurasia')]
[('p', 'http://www.wikidata.org/entity/Q620

In [76]:
#Using Europe to find all countries 
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
     wd:Q46 wdt:P527 ?p.
   # get the label
   ?p sc:name ?name.
   #FILTER(REGEX(?name,"Italy"^^xsd:string))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q263686'), ('name', 'Southeast Europe')]
[('p', 'http://www.wikidata.org/entity/Q27468'), ('name', 'Eastern Europe')]
[('p', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe')]
[('p', 'http://www.wikidata.org/entity/Q27509'), ('name', 'Central Europe')]
[('p', 'http://www.wikidata.org/entity/Q50807777'), ('name', 'Southwestern Europe')]
[('p', 'http://www.wikidata.org/entity/Q27449'), ('name', 'Southern Europe')]
[('p', 'http://www.wikidata.org/entity/Q27479'), ('name', 'Northern Europe')]
[('p', 'http://www.wikidata.org/entity/Q1998497'), ('name', 'North Eastern Europe')]
[('p', 'http://www.wikidata.org/entity/Q1414323'), ('name', 'Northwestern Europe')]
9


In [79]:
#Using Europe to find all countries 
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?p ?name ?cname ?m
WHERE {
   # bind something
     wd:Q46 wdt:P527 ?p.
     ?p wdt:P527 ?cname.
     
   # get the label
   ?p sc:name ?name.
   ?cname sc:name ?m.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q27449'), ('name', 'Southern Europe'), ('cname', 'http://www.wikidata.org/entity/Q1410'), ('m', 'Gibraltar')]
[('p', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe'), ('cname', 'http://www.wikidata.org/entity/Q142'), ('m', 'France')]
[('p', 'http://www.wikidata.org/entity/Q27509'), ('name', 'Central Europe'), ('cname', 'http://www.wikidata.org/entity/Q142'), ('m', 'France')]
[('p', 'http://www.wikidata.org/entity/Q27509'), ('name', 'Central Europe'), ('cname', 'http://www.wikidata.org/entity/Q145'), ('m', 'United Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q27479'), ('name', 'Northern Europe'), ('cname', 'http://www.wikidata.org/entity/Q145'), ('m', 'United Kingdom')]
[('p', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe'), ('cname', 'http://www.wikidata.org/entity/Q183'), ('m', 'Germany')]
[('p', 'http://www.wikidata.org/entity/Q27509'), ('name', 'Central Europe'), ('cname', 'http://www.wikida

In [292]:
#Using Europe to find all countries 
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?cname ?m 
WHERE {
   # bind something
     wd:Q46 wdt:P527 ?p.
     ?p wdt:P527 ?cname.
   # get the label
   #?pn sc:name ?pname.
   ?cname sc:name ?m.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('cname', 'http://www.wikidata.org/entity/Q1410'), ('m', 'Gibraltar')]
[('cname', 'http://www.wikidata.org/entity/Q142'), ('m', 'France')]
[('cname', 'http://www.wikidata.org/entity/Q145'), ('m', 'United Kingdom')]
[('cname', 'http://www.wikidata.org/entity/Q183'), ('m', 'Germany')]
[('cname', 'http://www.wikidata.org/entity/Q184'), ('m', 'Belarus')]
[('cname', 'http://www.wikidata.org/entity/Q189'), ('m', 'Iceland')]
[('cname', 'http://www.wikidata.org/entity/Q191'), ('m', 'Estonia')]
[('cname', 'http://www.wikidata.org/entity/Q20'), ('m', 'Norway')]
[('cname', 'http://www.wikidata.org/entity/Q211'), ('m', 'Latvia')]
[('cname', 'http://www.wikidata.org/entity/Q212'), ('m', 'Ukraine')]
[('cname', 'http://www.wikidata.org/entity/Q213'), ('m', 'Czech Republic')]
[('cname', 'http://www.wikidata.org/entity/Q214'), ('m', 'Slovakia')]
[('cname', 'http://www.wikidata.org/entity/Q215'), ('m', 'Slovenia')]
[('cname', 'http://www.wikidata.org/entity/Q217'), ('m', 'Moldova')]
[('cname', 

In [297]:
#Since we have all the countries from europe and Leagues 
#
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?lc ?lcname (COUNT(?l) AS ?NumOfLea) 
WHERE {
   # bind something
     wd:Q46 wdt:P527 ?p.
     ?p wdt:P527 ?cname.
     ?l ?obj wd:Q15991303;
         wdt:P17 ?lc.
   # get the label
   #?pn sc:name ?pname.
   #?cname sc:name ?m.
   ?lc sc:name ?lcname.
}
GROUP BY ?lc ?lcname 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('lc', 'http://www.wikidata.org/entity/Q218'), ('lcname', 'Romania'), ('NumOfLea', '4092')]
[('lc', 'http://www.wikidata.org/entity/Q28'), ('lcname', 'Hungary'), ('NumOfLea', '330')]
[('lc', 'http://www.wikidata.org/entity/Q754'), ('lcname', 'Trinidad and Tobago'), ('NumOfLea', '132')]
[('lc', 'http://www.wikidata.org/entity/Q865'), ('lcname', 'Taiwan'), ('NumOfLea', '198')]
[('lc', 'http://www.wikidata.org/entity/Q921'), ('lcname', 'Brunei'), ('NumOfLea', '66')]
[('lc', 'http://www.wikidata.org/entity/Q953'), ('lcname', 'Zambia'), ('NumOfLea', '132')]
[('lc', 'http://www.wikidata.org/entity/Q1044'), ('lcname', 'Sierra Leone'), ('NumOfLea', '66')]
[('lc', 'http://www.wikidata.org/entity/Q215'), ('lcname', 'Slovenia'), ('NumOfLea', '264')]
[('lc', 'http://www.wikidata.org/entity/Q222'), ('lcname', 'Albania'), ('NumOfLea', '198')]
[('lc', 'http://www.wikidata.org/entity/Q232'), ('lcname', 'Kazakhstan'), ('NumOfLea', '198')]
[('lc', 'http://www.wikidata.org/entity/Q424'), ('lcnam

Final query for this task

In [ ]:
# write your final query

## Task 5
Identify the BGP for the ballon d'Or award

In [92]:
#Using Ronaldos BGP to explore awards
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [93]:
#Using Award Recieved 
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P166 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1750244'), ('name', 'FWA Footballer of the Year')]
[('p', 'http://www.wikidata.org/entity/Q10726649'), ('name', "PFA Fans' Player of the Year")]
[('p', 'http://www.wikidata.org/entity/Q20888805'), ('name', 'Commander of the Order of Merit of Portugal')]
[('p', 'http://www.wikidata.org/entity/Q1146578'), ('name', 'UEFA Team of the Year')]
[('p', 'http://www.wikidata.org/entity/Q182529'), ('name', 'FIFA World Player of the Year')]
[('p', 'http://www.wikidata.org/entity/Q260117'), ('name', "UEFA Men's Player of the Year Award")]
[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
[('p', 'http://www.wikidata.org/entity/Q20860117'), ('name', 'Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q20859673'), ('name', 'Grand Officer of the Order of Prince Henry')]
[('p', 'http://www.wikidata.org/entity/Q2986505'), ('name', 'Sir Matt Busby Player of the Year')]
[('p', 'http://www.wikidata.org/entit

Final query for this task

In [94]:
# write your final query
#Using Ballon d'Or ID from Results
#[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q166177 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2572'), ('name', 'hashtag')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3222'), ('name', 'NE.se ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P64

## Task 6
Return the countries which have at least two different players of their nationality (consider the country of citizenship) who won the ballon d'Or (the result set must be a list of triples country IRI, label and #different players).

In [286]:
# Getting 2 IRIs from BGP of Ronaldo and Winner ID of Ballon d'Or award
#[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
#[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
queryString = """
SELECT DISTINCT ?p ?name  ?nname  ?cnname
WHERE {
   # bind something
   wd:Q166177 wdt:P1346 ?p.
   ?p wdt:P1532 ?n;
       wdt:P27 ?cn.
   # get the label
   ?p sc:name ?name.
   ?n sc:name ?nname.
   ?cn sc:name ?cnname.
   FILTER(?cnname = ?nname)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('nname', 'Germany'), ('cnname', 'Germany')]
[('p', 'http://www.wikidata.org/entity/Q483145'), ('name', 'Luís Figo'), ('nname', 'Portugal'), ('cnname', 'Portugal')]
[('p', 'http://www.wikidata.org/entity/Q39444'), ('name', 'Ronaldinho'), ('nname', 'Brazil'), ('cnname', 'Brazil')]
[('p', 'http://www.wikidata.org/entity/Q220700'), ('name', 'Omar Sívori'), ('nname', 'Argentina'), ('cnname', 'Argentina')]
[('p', 'http://www.wikidata.org/entity/Q13494'), ('name', 'Jean-Pierre Papin'), ('nname', 'France'), ('cnname', 'France')]
[('p', 'http://www.wikidata.org/entity/Q43744'), ('name', 'Gerd Müller'), ('nname', 'Germany'), ('cnname', 'Germany')]
[('p', 'http://www.wikidata.org/entity/Q152871'), ('name', 'Karl-Heinz Rummenigge'), ('nname', 'Germany'), ('cnname', 'Germany')]
[('p', 'http://www.wikidata.org/entity/Q1835'), ('name', 'Zinedine Zidane'), ('nname', 'France'), ('cnname', 'France')]
[('p', 'http://ww

Final query for this task

In [290]:
# write your final query
#[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
#[('p', 'http://www.wikidata.org/entity/Q166177'), ('name', "Ballon d'Or")]
queryString = """
SELECT DISTINCT (COUNT(distinct ?p) AS ?NumOfPl) ?cnname
WHERE {
   # bind something
   wd:Q166177 wdt:P1346 ?p.
   ?p wdt:P1532 ?n;
       wdt:P27 ?cn.
   # get the label
   ?p sc:name ?name.
   ?n sc:name ?nname.
   ?cn sc:name ?cnname.
   FILTER(?cnname != ?nname)
}
GROUP BY ?cnname
HAVING (COUNT(distinct ?p) >1)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfPl', '6'), ('cnname', 'United Kingdom')]
[('NumOfPl', '3'), ('cnname', 'Spain')]
[('NumOfPl', '2'), ('cnname', 'Soviet Union')]
[('NumOfPl', '2'), ('cnname', 'Kingdom of the Netherlands')]
[('NumOfPl', '2'), ('cnname', 'Italy')]
5


## Task 7
Identify the BGP of the FIFA World Cup

In [95]:
# Using Ronaldos BGP where he participated in
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/entity/Q102920'), ('name', 'UEFA Euro 2004')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q189571'), ('name', 'UEFA Euro 2016')]
[('p', 'http://www.wikidata.org/entity/Q22669'), ('name', 'UEFA Euro 2012')]
[('p', 'http://www.wikidata.org/entity/Q241864'), ('name', 'UEFA Euro 2008')]
[('p', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('p', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
11


In [96]:
#Using 2010 FIFA World Cup ID for exploring
#[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?obj ?p wd:Q176883.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P6291'), ('name', 'advertises')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P366'), ('name', 'use')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'm

In [98]:
#Using 2010 FIFA World Cup Has Part ID for exploring
#[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?p wdt:P527 wd:Q176883.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
1


Final query for this task

In [99]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q19317 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('name', 'WorldFootball.net competition ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('name', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('name', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct

## Task 8
Who are the players who partecipated in more than 4 world's cup editions? (the result set must be a list of triples player IRI, label and #participations).

In [237]:
#Using 2010 FIFA World Cup ID for exploring
#[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    wd:Q176883 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4967'), ('name', 'prerequisite')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http

In [157]:
#Exploring Teams Participated for getting teams
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    wd:Q176883 wdt:P1923 ?p.
    
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/entity/Q268208'), ('name', 'Australia national association football team')]
[('p', 'http://www.wikidata.org/entity/Q134916'), ('name', 'Uruguay national football team')]
[('p', 'http://www.wikidata.org/entity/Q170327'), ('name', 'South Africa national football team')]
[('p', 'http://www.wikidata.org/entity/Q14132'), ('name', 'North Korea national football team')]
[('p', 'http://www.wikidata.org/entity/Q174512'), ('name', 'Slovakia national association football team')]
[('p', 'http://www.wikidata.org/entity/Q175145'), ('name', 'Ivory Coast national football team')]
[('p', 'http://www.wikidata.org/entity/Q180187'), ('name', 'Honduras national football team')]
[('p', 'http://www.wikidata.org/entity/Q172221'), ('name', 'Slovenia national football team')]
[('p', 'http://www.wikidata.org/entity/Q47050'), ('name', 'Netherlands national association foo

In [236]:
#Getting Teams for All WCs

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?t.
    ?t wdt:P1923 ?p.
    
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q166196'), ('name', 'Poland national association football team')]
[('p', 'http://www.wikidata.org/entity/Q168397'), ('name', 'Romania national association football team')]
[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/entity/Q163534'), ('name', 'Austria national association football team')]
[('p', 'http://www.wikidata.org/entity/Q212564'), ('name', 'Colombia national football team')]
[('p', 'http://www.wikidata.org/entity/Q268208'), ('name', 'Australia national association football team')]
[('p', 'http://www.wikidata.org/entity/Q134916'), ('name', 'Uruguay national football team')]
[('p', 'http://www.wikidata.org/entity/Q181190'), ('name', 'Bosnia and Herzegovina national football team')]
[('p', 'http://www.wikidata.org/entity/Q170327'), ('name', 'South Africa national football team')]
[('p', 'http://www.wikidata.org/entity/Q186243'), ('name', 'Iraq n

In [181]:
#Exploring one country for relation to players
#[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
queryString = """
SELECT DISTINCT ?p ?name ?pl ?plname
WHERE {
   # bind something
    ?pl ?p wd:Q676899.
    
    
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?plname
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('pl', 'http://www.wikidata.org/entity/Q706023'), ('plname', '2004 UEFA European Under-21 Football Championship')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('pl', 'http://www.wikidata.org/entity/Q1875825'), ('plname', 'Guido Corbelli')]
2


In [197]:
#Exploring one country for relation to players
#[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
queryString = """
SELECT DISTINCT  ?pl ?plname
WHERE {
   # bind something
    ?pl wdt:P54 wd:Q676899.
    
    
   # get the label
   #?p sc:name ?name.
   ?pl sc:name ?plname
}
LIMIT 2

"""

print("Results")
x=run_query(queryString)

Results
[('pl', 'http://www.wikidata.org/entity/Q1875825'), ('plname', 'Guido Corbelli')]
[('pl', 'http://www.wikidata.org/entity/Q3092596'), ('plname', 'Felice Romano')]
2


In [199]:
#Adding relation member of sports team

queryString = """
SELECT DISTINCT ?p ?name ?pl ?plname ?t ?tname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?t.
    ?t wdt:P1923 ?p.
     ?pl wdt:P54 ?p.
         
    
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?plname.
   ?t sc:name ?tname.
      FILTER(REGEX(?plname,"Lionel Messi"^^xsd:string))

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team'), ('pl', 'http://www.wikidata.org/entity/Q615'), ('plname', 'Lionel Messi'), ('t', 'http://www.wikidata.org/entity/Q101751'), ('tname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team'), ('pl', 'http://www.wikidata.org/entity/Q615'), ('plname', 'Lionel Messi'), ('t', 'http://www.wikidata.org/entity/Q48432'), ('tname', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team'), ('pl', 'http://www.wikidata.org/entity/Q615'), ('plname', 'Lionel Messi'), ('t', 'http://www.wikidata.org/entity/Q176883'), ('tname', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79800'), ('name', 'Argentina national football team'), ('pl', 'http://www.wikidata.org/entity/Q615'), ('plname', 'Lionel Messi'), ('t', 'http://www.wikidata.org/entity/Q46938'), ('tname', '1986 

In [193]:
#Getting Teams for All WCs

queryString = """
SELECT DISTINCT (COUNT(?t) as ?NumOfWC) ?pl ?plname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?t.
    ?t wdt:P1923 ?p.
     ?pl wdt:P54 ?p.
         
    
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?plname
   FILTER(REGEX(?plname,"Messi"^^xsd:string))
}
GROUP BY ?plname ?pl
ORDER BY DESC(?NumOfWC)
LIMIT 200

"""

print("Results")
x=run_query(queryString)

Results
[('NumOfWC', '16'), ('pl', 'http://www.wikidata.org/entity/Q615'), ('plname', 'Lionel Messi')]
[('NumOfWC', '8'), ('pl', 'http://www.wikidata.org/entity/Q17364536'), ('plname', 'Bruno Valdez Messi')]
[('NumOfWC', '6'), ('pl', 'http://www.wikidata.org/entity/Q3307048'), ('plname', 'Messias Timula')]
[('NumOfWC', '2'), ('pl', 'http://www.wikidata.org/entity/Q9028594'), ('plname', 'Marcelo Messias')]
4


In [214]:
#Probably a Wrong / difficult Approach to filter out players as we see that Query is finding number of worldcups by each time and regardless if he played worldcup or not his name is displayed
#Using Football Association Player ID to use the other approach
# BGP of Association Football Player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q937857 ?obj ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P2369'), ('name', 'Soccerway player ID')]
[('p', 'http://www.wikidata.org/entity/P3908'), ('name', 'Reprezentacija player ID')]
[('p', 'http://www.wikidata.org/entity/Q18536342'), ('name', 'competitive player')]
[('p', 'http://www.wikidata.org/entity/P2446'), ('name', 'Transfermarkt player ID')]
[('p', 'http://www.wikidata.org/entity/Q7217464'), ('name', 'Category:Association football players')]
[('p', 'http://www.wikidata.org/entity/P2823'), ('name', 'Royal Belgian Football Association player ID')]
[('p', 'http://www.wikidata.org/entity/Q15944511'), ('name', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('p', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('p', 'http://www.wikidata.org/entity/Q476028'), ('name', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q31930761'), ('name', "men's association football")]
[('p', 'http://www.wi

In [234]:
#Getting FIFA Player ID
#[('p', 'http://www.wikidata.org/entity/P1469'), ('name', 'FIFA player ID')]
queryString = """
SELECT DISTINCT ?p ?name ?pl ?pname
WHERE {
   # bind something
    ?p wdt:P1469 ?obj;
        wdt:P1344 ?pl.
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?pname.
   FILTER(REGEX(?pname,"FIFA World Cup$"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1652615'), ('name', 'Paul Caligiuri'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q261069'), ('name', 'Stig Inge Bjørnebye'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q448678'), ('name', 'Trifon Ivanov'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q178649'), ('name', 'Romário'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q486648'), ('name', 'Gu Sang-bum'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q486937'), ('name', 'Choi In-young'), ('pl', 'http://www.wikidata.org/entity/Q101751'), ('pname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidat

In [235]:
#Counting Appearance of players
#[('p', 'http://www.wikidata.org/entity/P1469'), ('name', 'FIFA player ID')]
queryString = """
SELECT DISTINCT ?p ?name (COUNT(distinct ?pl) AS ?NumOfWC)
WHERE {
   # bind something
    ?p wdt:P1469 ?obj;
        wdt:P1344 ?pl.
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?pname.
   FILTER(REGEX(?pname,"FIFA World Cup$"^^xsd:string))
}
GROUP BY ?p ?name 
ORDER BY DESC(?NumOfWC)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q186330'), ('name', 'Rafael Márquez'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q265428'), ('name', 'Antonio Carbajal'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q68060'), ('name', 'Gianluigi Buffon'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q299693'), ('name', 'Djalma Santos'), ('NumOfWC', '4')]
[('p', 'http://www.wikidata.org/entity/Q274872'), ('name', 'Émerson Leão'), ('NumOfWC', '4')]
[('p', 'http://www.wikidata.org/entity/Q161571'), ('name', 'Javier Mascherano'), ('NumOfWC', '4')]
[('p', 'http://www.wikidata.org/entity/Q80471'), ('name', 'Miroslav Klose'), ('NumOfWC', '4')]
[('p', 'http://www.wikidata.org/entity/Q312179'), ('name', 'Mohamed Al-Deayea'), ('NumOfWC', '4')]
[('p', 'http://www.wikidata.org/entity/Q313035'), ('name', 'DaMarcus Beasley'), ('NumOfWC', '4')]
[('p', 'http://www.wikida

Final query for this task

In [240]:
# write your final query
#Players Who Participated in more than 4 World Cups (Data doesnot include 2022 Worldcup)
#Counting Appearance of players
#[('p', 'http://www.wikidata.org/entity/P1469'), ('name', 'FIFA player ID')]
queryString = """
SELECT DISTINCT ?p ?name (COUNT(distinct ?pl) AS ?NumOfWC)
WHERE {
   # bind something
    ?p wdt:P1469 ?obj;
        wdt:P1344 ?pl.
   # get the label
   ?p sc:name ?name.
   ?pl sc:name ?pname.
   FILTER(REGEX(?pname,"FIFA World Cup$"^^xsd:string))
}
GROUP BY ?p ?name 
HAVING (COUNT(distinct ?pl) > 4)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q186330'), ('name', 'Rafael Márquez'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q265428'), ('name', 'Antonio Carbajal'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q36386'), ('name', 'Lothar Matthäus'), ('NumOfWC', '5')]
[('p', 'http://www.wikidata.org/entity/Q68060'), ('name', 'Gianluigi Buffon'), ('NumOfWC', '5')]
4


## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the players of those countries won the ballon d'Or (the result set must be a list of couples player IRI and label).

In [241]:
# Using FIFA WC BGP to explore
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   # get the label
   ?p sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q1477177'), ('name', '1934 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q203851'), ('name', '1942 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q65273310'), ('name', '1946 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132515'), ('name', '1950 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131075'), ('name', '1954 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132533'), ('name', '1958 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q160813'), ('name', '1962 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q134202'), ('name', '1966 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132664'), ('name', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q166121'), ('name', '1974 FIFA World C

In [106]:
#We know from Task 7 we have winner in each FIFA WC
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT ?p ?name ?c ?cname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   ?p wdt:P1346 ?c.
   # get the label
   ?p sc:name ?name.
   ?c sc:name ?cname.
}
ORDER BY ASC(?name)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q134916'), ('cname', 'Uruguay national football team')]
[('p', 'http://www.wikidata.org/entity/Q1477177'), ('name', '1934 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/entity/Q132515'), ('name', '1950 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q134916'), ('cname', 'Uruguay national football team')]
[('p', 'http://www.wikidata.org/entity/Q131075'), ('name', '1954 FIFA World Cup'), ('c', 'http://www.wikidata.org/entity/Q43310'), ('cname', 'Germany national association football team')]
[('p', 'http://www.wikidata.org/entity/Q132533'), ('name', '1958 FIFA World 

In [131]:
#Counting Number of World Cups Won
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfWCs) ?c ?cname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   ?p wdt:P1346 ?c.
   # get the label
   ?p sc:name ?name.
   ?c sc:name ?cname.
}
GROUP BY ?c ?cname
HAVING (COUNT(?p) > 1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q47774'), ('cname', 'France national association football team')]
[('NumOfWCs', '5'), ('c', 'http://www.wikidata.org/entity/Q83459'), ('cname', 'Brazil national football team')]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q134916'), ('cname', 'Uruguay national football team')]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q79800'), ('cname', 'Argentina national football team')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q43310'), ('cname', 'Germany national association football team')]
6


In [132]:
#Now we have all the countries who won world cups atleast twice 
#So now we will explore their players
#Taking one specific country to find relation between Teams and Players
#('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team')]
queryString = """
SELECT DISTINCT ?p ?name ?aw ?aname
WHERE {
   # bind something
      ?p ?obj wd:Q676899;
          wdt:P166 ?aw.
   # get the label
   ?p sc:name ?name.
   ?aw sc:name ?aname
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q43304'), ('name', 'Marcello Lippi'), ('aw', 'http://www.wikidata.org/entity/Q1847685'), ('aname', 'Italian Football Hall of Fame')]
[('p', 'http://www.wikidata.org/entity/Q313457'), ('name', 'Enzo Bearzot'), ('aw', 'http://www.wikidata.org/entity/Q1847685'), ('aname', 'Italian Football Hall of Fame')]
2


In [242]:
#Counting Number of World Cups Won
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfWCs) ?c ?cname ?jname ?aname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   ?p wdt:P1346 ?c.
   
   
   ?j ?obj ?c;
          wdt:P166 ?aw.
   # get the label
   ?j sc:name ?jname.
   ?aw sc:name ?aname.
   # get the label
   ?p sc:name ?name.
   ?c sc:name ?cname.
}
GROUP BY ?c ?cname ?jname ?aname
HAVING (COUNT(?p) > 1)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname', 'Fabio Cannavaro'), ('aname', 'Officer of the Order of Merit of the Italian Republic')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname', 'Fabio Cannavaro'), ('aname', 'FIFA World Player of the Year')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname', 'Federico Chiesa'), ('aname', 'Knight of the Order of Merit of the Italian Republic')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname', 'Ciro Ferrara'), ('aname', 'Knight of the Order of Merit of the Italian Republic')]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname',

In [134]:
#Specifying Award
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfWCs) ?c ?cname ?jname ?aname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   ?p wdt:P1346 ?c.
   
   
   ?j ?obj ?c;
          wdt:P166 ?aw.
   # get the label
   ?j sc:name ?jname.
   ?aw sc:name ?aname.
   # get the label
   ?p sc:name ?name.
   ?c sc:name ?cname.
   FILTER(REGEX(?aname,"Ballon d'Or"^^xsd:string))
}
GROUP BY ?c ?cname ?jname ?aname
HAVING (COUNT(?p) > 1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q79800'), ('cname', 'Argentina national football team'), ('jname', 'Lionel Messi'), ('aname', "FIFA Ballon d'Or")]
[('NumOfWCs', '4'), ('c', 'http://www.wikidata.org/entity/Q676899'), ('cname', 'Italy national association football team'), ('jname', 'Paolo Rossi'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q47774'), ('cname', 'France national association football team'), ('jname', 'Michel Platini'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q47774'), ('cname', 'France national association football team'), ('jname', 'Zinedine Zidane'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q47774'), ('cname', 'France national association football team'), ('jname', 'Raymond Kopa'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('c', 'http://www.wikidata.org/entity/Q79800'), ('cname', 'Argentina national football 

Final query for this task

In [140]:
# write your final query
queryString = """
SELECT DISTINCT (COUNT(?p) AS ?NumOfWCs) ?cname ?jname ?aname
WHERE {
   # bind something
   wd:Q19317 wdt:P527 ?p.
   ?p wdt:P1346 ?c.
   
   
   ?j ?obj ?c;
          wdt:P166 ?aw.
   # get the label
   ?j sc:name ?jname.
   ?aw sc:name ?aname.
   # get the label
   ?p sc:name ?name.
   ?c sc:name ?cname.
   FILTER(REGEX(?aname,"Ballon d'Or"^^xsd:string))
}
GROUP BY ?c ?cname ?jname ?aname
HAVING (COUNT(?p) > 1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfWCs', '2'), ('cname', 'Argentina national football team'), ('jname', 'Lionel Messi'), ('aname', "FIFA Ballon d'Or")]
[('NumOfWCs', '4'), ('cname', 'Italy national association football team'), ('jname', 'Paolo Rossi'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('cname', 'France national association football team'), ('jname', 'Michel Platini'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('cname', 'France national association football team'), ('jname', 'Zinedine Zidane'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('cname', 'France national association football team'), ('jname', 'Raymond Kopa'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '2'), ('cname', 'Argentina national football team'), ('jname', 'Lionel Messi'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '4'), ('cname', 'Germany national association football team'), ('jname', 'Lothar Matthäus'), ('aname', "Ballon d'Or")]
[('NumOfWCs', '4'), ('cname', 'Germany national association football team'), ('jname', 'Gerd Mü